In [5]:
# Modified test_specific_inputs.py with visualization
import numpy as np
import logging
import matplotlib.pyplot as plt
from trainer import CompilerOptimizationTrainer
from model_validator import ModelValidator

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def test_specific_configurations():
    """Test model on predefined compiler configurations and visualize results"""
    
    # Initialize trainer
    trainer = CompilerOptimizationTrainer("data/exec_times.csv")
    trainer.setup()
    
    # Load trained models if available
    import os
    models_dir = "models"
    if os.path.exists(models_dir):
        for benchmark in trainer.benchmark_data.keys():
            model_path = f"{models_dir}/{benchmark}_final.pth"
            if os.path.exists(model_path):
                trainer.agents[benchmark].load(model_path)
                logger.info(f"Loaded model for {benchmark}")
    
    # Create validator
    validator = ModelValidator(trainer)
    
    # Define test configurations
    test_configs = [
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 1, 0, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 0, 1, 0, 1, 0, 1],
        [1, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 0, 0]
    ]
    
    config_names = [
        "No opts", "Only O2", "Aggressive", "Conservative", 
        "All flags", "Mixed", "Unsafe math", "Disable opts"
    ]
    
    # Test on first available benchmark
    benchmark_names = list(trainer.benchmark_data.keys())
    
    for benchmark in benchmark_names[:2]:
        print(f"\n{'='*60}")
        print(f"Testing configurations on benchmark: {benchmark}")
        print(f"{'='*60}")
        
        # Test specific configurations
        results = validator.test_specific_configurations(benchmark, test_configs)
        
        if results:
            print(f"\nResults for {benchmark}:")
            print("-" * 50)
            
            # Collect data for visualization
            input_improvements = []
            suggested_improvements = []
            for i, (config_name, result) in enumerate(zip(config_names, results['results'])):
                input_config = result['input_config']
                suggested_config = result['suggested_config']
                actual_improvement = result['actual_improvement'] * 100
                suggested_improvement = result['suggested_improvement'] * 100
                model_improvement = result['model_improvement'] * 100
                
                print(f"\n{config_name}:")
                print(f"  Input config:     {input_config}")
                print(f"  Suggested config: {suggested_config}")
                print(f"  Input improvement:     {actual_improvement:6.2f}%")
                print(f"  Suggested improvement: {suggested_improvement:6.2f}%")
                print(f"  Model improvement:     {model_improvement:+6.2f}%")
                
                if result['config_changed']:
                    print(f"  ✓ Model suggested different configuration")
                else:
                    print(f"  - Model kept same configuration")
                
                input_improvements.append(actual_improvement)
                suggested_improvements.append(suggested_improvement)
            
            # Print summary
            summary = results['summary']
            print(f"\nSummary for {benchmark}:")
            print(f"  Total configurations tested: {summary['total_configs_tested']}")
            print(f"  Configurations improved: {summary['configs_improved']}")
            print(f"  Improvement rate: {summary['improvement_rate']:.4f}")
            print(f"  Mean improvement: {summary['mean_improvement']*100:.2f}%")
            print(f"  Mean positive improvement: {summary['mean_positive_improvement']*100:.2f}%")
            print(f"  Maximum improvement: {summary['max_improvement']*100:.2f}%")
            print(f"  Minimum improvement: {summary['min_improvement']*100:.2f}%")
            
            # Visualize improvements
            fig, ax = plt.subplots(figsize=(10, 6))
            x = np.arange(len(config_names))
            width = 0.35
            
            ax.bar(x - width/2, input_improvements, width, label='Input Improvement', alpha=0.6)
            ax.bar(x + width/2, suggested_improvements, width, label='Suggested Improvement', alpha=0.6)
            
            ax.set_xlabel('Configuration')
            ax.set_ylabel('Improvement (%)')
            ax.set_title(f'Improvement Comparison for {benchmark}')
            ax.set_xticks(x)
            ax.set_xticklabels(config_names, rotation=45, ha='right')
            ax.legend()
            ax.grid(True, alpha=0.3)
            
            plt.tight_layout()
            os.makedirs("plots", exist_ok=True)
            plt.savefig(f"plots/{benchmark}_config_improvements.png", dpi=300, bbox_inches='tight')
            plt.show()

if __name__ == "__main__":
    test_specific_configurations()

ModuleNotFoundError: No module named 'seaborn'

In [2]:
!pip install seaborn


  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
  Using cached matplotlib-3.9.4-cp39-cp39-win_amd64.whl (7.8 MB)
  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl (15.9 MB)
  Using cached pandas-2.3.0-cp39-cp39-win_amd64.whl (11.1 MB)
  Using cached kiwisolver-1.4.7-cp39-cp39-win_amd64.whl (55 kB)
  Using cached fonttools-4.58.4-cp39-cp39-win_amd64.whl (1.5 MB)
  Using cached pillow-11.2.1-cp39-cp39-win_amd64.whl (2.7 MB)
  Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
  Using cached contourpy-1.3.0-cp39-cp39-win_amd64.whl (211 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl (37 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached packaging-25.0-py3-none-any.whl (66 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  Using cached zipp-3.23.0-py3-none-any.whl (10 kB)
  Using cached six-1.17.0-p

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\sis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.
